In [ ]:
import cv2
import time
import json
import torch
import gc
import numpy as np
import pandas as pd
from pathlib import Path
from sam3.model_builder import build_sam3_video_model

# ====================================================
# 1. 설정 및 경로
# ====================================================
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
CSV_PATH = DATA_DIR / "metadata.csv"

# 메타데이터 로드
df = pd.read_csv(CSV_PATH)
target = 1  
COMMON_PATH = df.loc[target, "common_path"]
VIDEO_DIR = DATA_DIR / "test" / "frame_change"
KPT_DIR = DATA_DIR / "2_KEYPOINTS" / COMMON_PATH
CHECKPOINT_PT = DATA_DIR / "checkpoints/SAM3/sam3.pt"

# 결과 저장 폴더
OUTPUT_RESULT_DIR = DATA_DIR / "test" / "benchmark_overlay_result"
OUTPUT_RESULT_DIR.mkdir(parents=True, exist_ok=True)

print(f"🚀 벤치마킹 & 시각화 시작: {COMMON_PATH}")
print(f"📂 결과 저장 경로: {OUTPUT_RESULT_DIR}")

# ====================================================
# 2. 유틸리티 함수: 마스크 오버레이
# ====================================================
def apply_mask_overlay(image, mask, color=(0, 255, 0), alpha=0.5):
    """이미지에 이진 마스크를 반투명하게 덮어씌웁니다."""
    if mask.ndim != 2: return image
    
    overlay = image.copy()
    overlay[mask > 0] = color
    
    output = image.copy()
    output[mask > 0] = cv2.addWeighted(image, 1-alpha, overlay, alpha, 0)[mask > 0]
    
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(output, contours, -1, (255, 255, 255), 1)
    
    return output

# ====================================================
# 3. 초기 설정 (BBox 로드 및 모델 준비)
# ====================================================
REF_W, REF_H = 1280, 720 # Keypoint 기준 해상도

with open(KPT_DIR / "000000.json", 'r') as f:
    kpt_data = json.load(f)

raw_bbox = kpt_data['instance_info'][0]['bbox'][0]
rel_bbox = [
    raw_bbox[0] / REF_W,
    raw_bbox[1] / REF_H,
    raw_bbox[2] / REF_W,
    raw_bbox[3] / REF_H
]

print(f"\n[Init] SAM 3 모델 로드 중...")
sam3_model = build_sam3_video_model(checkpoint_path=CHECKPOINT_PT)
predictor = sam3_model.tracker
predictor.backbone = sam3_model.detector.backbone

# ====================================================
# 4. 벤치마킹 루프
# ====================================================
video_files = list(VIDEO_DIR.glob("*.mp4"))
results = []

print(f"\n[Start] 총 {len(video_files)}개 영상 테스트 시작")
print("-" * 60)

for vid_path in sorted(video_files, reverse=True):
    # 비디오 정보 읽기
    cap = cv2.VideoCapture(str(vid_path))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"👉 Testing: {vid_path.name} ({width}x{height}, {total_frames} frames)")
    
    save_name = OUTPUT_RESULT_DIR / f"overlay_{vid_path.name}"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(save_name), fourcc, fps, (width, height))
    
    # 메모리 정리 (OOM 방지)
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    gc.collect()
    
    try:
        # 1. 초기화
        inference_state = predictor.init_state(
            video_path=str(vid_path),
            offload_video_to_cpu=True,
            async_loading_frames=True
        )
        predictor.clear_all_points_in_video(inference_state)
        
        # 2. 프롬프트
        abs_box = np.array([[
            rel_bbox[0] * width, rel_bbox[1] * height,
            rel_bbox[2] * width, rel_bbox[3] * height
        ]], dtype=np.float32)
        
        predictor.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=0,
            obj_id=1,
            box=abs_box
        )
        
        # 3. 추론 및 저장 루프
        pure_inference_time = 0
        frame_cnt = 0
        
        iterator = predictor.propagate_in_video(
            inference_state, 
            start_frame_idx=0,
            max_frame_num_to_track=total_frames,
            reverse=False,
            propagate_preflight=True
        )
        
        while True:
            t0 = time.time()
            try:
                step_result = next(iterator)
                
                if len(step_result) >= 4:
                    out_frame_idx = step_result[0]
                    out_obj_ids = step_result[1]
                    out_mask_logits = step_result[3] 
                else:
                    out_frame_idx, out_obj_ids, out_mask_logits = step_result[:3]

            except StopIteration:
                break
            except ValueError as ve: 
                print(f"⚠️ 리턴값 언패킹 에러: {ve}")
                break

            pure_inference_time += (time.time() - t0)
            frame_cnt += 1
            
            # --- 시각화 ---
            ret, frame = cap.read()
            if not ret: break
            
            for i, obj_id in enumerate(out_obj_ids):
                if obj_id == 1:
                    mask = (out_mask_logits[i] > 0.0).cpu().numpy().squeeze()
                    if mask.ndim == 2:
                        frame = apply_mask_overlay(frame, mask, color=(0, 255, 0), alpha=0.6)
            
            cv2.putText(frame, f"SAM 3 | {width}x{height}", (20, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            out.write(frame)
            # -------------

        cap.release()
        out.release()
        
        fps = frame_cnt / pure_inference_time if pure_inference_time > 0 else 0
        print(f"   ✅ 완료! Inf Time: {pure_inference_time:.2f}s | FPS: {fps:.2f}")
        print(f"   💾 영상 저장: {save_name.name}")
        
        results.append({
            "File": vid_path.name,
            "Resolution": f"{width}x{height}",
            "Inference Time(s)": round(pure_inference_time, 2),
            "FPS": round(fps, 2),
            "Frames": frame_cnt
        })
        
        # 💡 [수정됨] reset_state 제거, del로만 정리
        del inference_state
        
    except Exception as e:
        print(f"   ❌ 에러 발생: {e}")
        import traceback
        traceback.print_exc() 
        cap.release()
        out.release()

# ====================================================
# 5. 리포트 저장
# ====================================================
df_res = pd.DataFrame(results).sort_values("FPS", ascending=True)
print("\n" + "="*60)
print(df_res)
df_res.to_csv(OUTPUT_RESULT_DIR / "benchmark_overlay_report.csv", index=False)

🚀 벤치마킹 & 시각화 시작: AI_dataset/N01/N01_Treatment/diagonal__hip_extension
📂 결과 저장 경로: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/test/benchmark_overlay_result

[Init] SAM 3 모델 로드 중...


INFO 2025-12-18 12:07:08,560 121 sam3_video_base.py: 124: setting max_num_objects=10000 and num_obj_for_compile=16



[Start] 총 4개 영상 테스트 시작
------------------------------------------------------------
👉 Testing: diagonal__hip_extension_640x360.mp4 (640x360, 320 frames)


propagate in video: 100% 320/320 [06:52<00:00,  1.29s/it]


   ✅ 완료! Inf Time: 361.58s | FPS: 0.88
   💾 영상 저장: overlay_diagonal__hip_extension_640x360.mp4
👉 Testing: diagonal__hip_extension_480x270.mp4 (480x270, 320 frames)


...

   ✅ 완료! Inf Time: 363.98s | FPS: 0.88
   💾 영상 저장: overlay_diagonal__hip_extension_480x270.mp4
👉 Testing: diagonal__hip_extension_1280x720.mp4 (1280x720, 320 frames)


propagate in video: 100% 320/320 [06:28<00:00,  1.21s/it]


   ✅ 완료! Inf Time: 363.66s | FPS: 0.88
   💾 영상 저장: overlay_diagonal__hip_extension_1280x720.mp4
👉 Testing: diagonal__hip_extension_1024x576.mp4 (1024x576, 320 frames)


propagate in video: 100% 320/320 [06:53<00:00,  1.29s/it]

   ✅ 완료! Inf Time: 360.36s | FPS: 0.89
   💾 영상 저장: overlay_diagonal__hip_extension_1024x576.mp4

                                   File Resolution  Inference Time(s)   FPS  \
0   diagonal__hip_extension_640x360.mp4    640x360             361.58  0.88   
1   diagonal__hip_extension_480x270.mp4    480x270             363.98  0.88   
2  diagonal__hip_extension_1280x720.mp4   1280x720             363.66  0.88   
3  diagonal__hip_extension_1024x576.mp4   1024x576             360.36  0.89   

   Frames  
0     320  
1     320  
2     320  
3     320  
